Notes on usage:

- Make sure to [change runtime to GPU](https://www.tutorialspoint.com/google_colab/google_colab_using_free_gpu.htm).
- The transcript will be saved in Files, which you can find in the menu on the left.
- Change the number of speakers below if different from two.
- Pick a bigger model if you want more accuracy and a smaller model if you want the program to run faster ([more info](https://github.com/openai/whisper#available-models-and-languages)).
- If you know the language being spoken is English, then change language to 'English' as this improves performance.


High level overview of what's happening here:


1.   I'm using Open AI's Whisper model to seperate audio into segments and generate transcripts.
2.   I'm then generating speaker embeddings for each segments.
3.   Then I'm using agglomerative clustering on the embeddings to identify the speaker for each segment.

Let me know if I can make it better!


V7

### Realidad
Los clips se analizaron uno por uno y se agrego los cortes a una lista temporal que luego se le pasaran al cluster. y los segmentos con los datos se actualizaron sus inicios de tiempo segun la posicion del clip y se agregaron a una agrupacion de segmentos. Luego se procesa el cluster con todos los miniclips y se asignan los sujetos. Luego se separo la generacion del texto para poder separarlo.

Varias formas de mejorar: Opciones
1. (yes) Acumular las transcripciones en una sola lista y pasarlo despues al archivo acumulado, acumular el entrenamiento del cluster, mejor entendimiento del codigo
2. Agregar un par de contadores para sincronizar la lista completa con speakers y los clips para unirlo, acumular el entrenamiento del cluster, menos memoria para mantener las listas

Importantes cambios:
- (yes) Mantener el cluster entre clips, para mantener el mismo sujeto a lo largo
- (yes) Unir los clips y (no) colocar alguna marca caracteristica
- (no) Generar un archivo HTML??
- (no) Reducir datos inncesarios
- (yes) auto download: https://www.youtube.com/watch?v=Zsbo959LP6A&ab_channel=NonoMart%C3%ADnezAlonso


# V8

## Tareas a realizar
1. En el audio2 parece ver un retraso con algunas secciones del audio 14:36, 14:53
  1. Realizar un html-debug que ayude a identificar inicios segementos y final de estos con detalle
  ```html
  <span style="color: red;">
    [${start}:${end}] ${text}
  </span>
  ```

# V9

Mejoras:
* Fijar version
* Ordenar ejecucion (Pendiente)


In [ ]:
path = "audio.wav"

In [ ]:
# upload audio file

from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving audio 231205.dat to audio 231205.dat


In [ ]:
#!pip uninstall torch torchvision torchaudio pyannote.core pyannote.database torchdata semver soundfile einops pytorch-metric-learning
#!pip install torch==2.1.0 torchvision==0.16.0+cu118
#!pip install triton==2.0.0
!pip uninstall torch torchvision torchaudio torchdata torchtext triton -y
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 torchtext==0.15.2  triton==2.0.0 torchdata==0.6.1
!pip install openai cohere tiktoken
print("p1")
#!pip install -q git+https://github.com/openai/whisper.git@v20230918 > /dev/null
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
print("p2")
#!pip install -q git+https://github.com/pyannote/pyannote-audio@b7960890aefda70e9595609d52e61e7e61fb16d7 > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
print("p3")
!pip install PyDub

Found existing installation: torch 2.1.0+cu118
Uninstalling torch-2.1.0+cu118:
  Successfully uninstalled torch-2.1.0+cu118
Found existing installation: torchvision 0.16.0+cu118
Uninstalling torchvision-0.16.0+cu118:
  Successfully uninstalled torchvision-0.16.0+cu118
Found existing installation: torchaudio 2.1.0+cu118
Uninstalling torchaudio-2.1.0+cu118:
  Successfully uninstalled torchaudio-2.1.0+cu118
Found existing installation: torchdata 0.7.0
Uninstalling torchdata-0.7.0:
  Successfully uninstalled torchdata-0.7.0
Found existing installation: torchtext 0.16.0
Uninstalling torchtext-0.16.0:
  Successfully uninstalled torchtext-0.16.0
Found existing installation: triton 2.1.0
Uninstalling triton-2.1.0:
  Successfully uninstalled triton-2.1.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 70.8 MB/s eta 0:00:00
   

In [ ]:
!pip install lightning-fabric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 4.8 MB/s eta 0:00:00


In [ ]:

nombre_archivo = path

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [ ]:
version_tscript = 9 #@param {type: "integer"}

num_speakers = 2 #@param {type:"integer"}

language = 'any' #@param ['any', 'English', 'es']

model_size = 'large-v2' #@param ['tiny', 'base', 'small', 'medium', 'large', 'large-v2']

clips_time_min = 10 #@param {type: "integer"}
direction_clips = 'clips/' #@param {type:"string"}

direction_text = 'texts/' #@param {type:"string"}

model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


In [ ]:
!rm clips/*
!rm clips
!mkdir clips
!rm texts/*
!rm texts
!mkdir texts

#import wave
#import contextlib

if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

with contextlib.closing(wave.open(path,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)
    duration_total = duration

#!pip install pydub

from pydub import AudioSegment

song = AudioSegment.from_file("audio.wav", format="wav")
#with AudioSegment.from_file("audio.wav", format="wav") as song:
# PyDub handles time in milliseconds
seg = 1000

clips = []
while len(clips) * clips_time_min * 60 < duration:
  len_clips = len(clips)
  clips.append({
      "start": len_clips * clips_time_min * 60 * seg,
      "end": min((len_clips+1) * clips_time_min * 60 * seg, duration * seg),
      "file-clip": direction_clips+"clip-"+str(len_clips)+".wav",
      "file-txt": direction_text+"part-"+str(len_clips)+".txt"
  })


for i, clip in enumerate(clips):

  print("init cut clip " + str(i))
  print(clip)
  start = clip["start"]
  end = clip["end"]
  name_clip = clip["file-clip"]
  clip_sound = song[start:end]
  #with song[start:end] as clip_sound:
  clip_sound.export(name_clip, format="wav")
  print(name_clip)

song = None
clip_sound = None

rm: cannot remove 'clips/*': No such file or directory
rm: cannot remove 'clips': No such file or directory
rm: cannot remove 'texts/*': No such file or directory
rm: cannot remove 'texts': No such file or directory
init cut clip 0
{'start': 0, 'end': 600000, 'file-clip': 'clips/clip-0.wav', 'file-txt': 'texts/part-0.txt'}
clips/clip-0.wav
init cut clip 1
{'start': 600000, 'end': 1200000, 'file-clip': 'clips/clip-1.wav', 'file-txt': 'texts/part-1.txt'}
clips/clip-1.wav
init cut clip 2
{'start': 1200000, 'end': 1800000, 'file-clip': 'clips/clip-2.wav', 'file-txt': 'texts/part-2.txt'}
clips/clip-2.wav
init cut clip 3
{'start': 1800000, 'end': 2400000, 'file-clip': 'clips/clip-3.wav', 'file-txt': 'texts/part-3.txt'}
clips/clip-3.wav
init cut clip 4
{'start': 2400000, 'end': 3000000, 'file-clip': 'clips/clip-4.wav', 'file-txt': 'texts/part-4.txt'}
clips/clip-4.wav
init cut clip 5
{'start': 3000000, 'end': 3600000, 'file-clip': 'clips/clip-5.wav', 'file-txt': 'texts/part-5.txt'}
clips/clip-

In [ ]:
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 2.87G/2.87G [00:47<00:00, 65.6MiB/s]


In [ ]:
def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

def time(secs):
  return datetime.timedelta(seconds=round(secs))



segments_group = []
segment_embedding_group = []
for y, clip in enumerate(clips):
  print("Clip "+ str(y)+ " - " + str(y+1)+"/"+str(len(clips)) + " workinggg....")
  path = clip["file-clip"]

  result = model.transcribe(path)
  segments = result["segments"]

  with contextlib.closing(wave.open(path,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)
  audio = Audio()
  for i, segment in enumerate(segments):
    segment_embedding_group.append(segment_embedding(segment))
  for i in range(len(segments)):
    segments[i]["start"] += clips_time_min*y*60
    segments[i]["end"] += clips_time_min*y*60
  segments_group += segments


embeddings = np.zeros(shape=(len(segments_group), 192))
for i, segment in enumerate(segments_group):
  embeddings[i] = segment_embedding_group[i]

embeddings = np.nan_to_num(embeddings)
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments_group)):
  segments_group[i]["speaker"] = 'SPEAKER' + str(labels[i] + 1)







Clip 0 - 1/9 workinggg....
Clip 1 - 2/9 workinggg....
Clip 2 - 3/9 workinggg....
Clip 3 - 4/9 workinggg....
Clip 4 - 5/9 workinggg....
Clip 5 - 6/9 workinggg....
Clip 6 - 7/9 workinggg....
Clip 7 - 8/9 workinggg....
Clip 8 - 9/9 workinggg....


# Text Generator

In [ ]:
ft = open(direction_text+"translation.txt", "w", encoding='utf-8')
ft.write('Atencion: Documento transcrito por secciones e IA, Revisar\n')
ft.write('Datos:\n')
ft.write(f'- Version Transcriptior: v{version_tscript}\n')
ft.write('- Nombre archivo original: '+ nombre_archivo +'\n')
ft.write('- Duracion de la grabacion: '+ str(time(duration_total)) +'\n')
ft.write('- Duracion maxima de cada clip: '+ str(time(clips_time_min * 60)) +'\n')
ft.write('- Tamaño del modelo de Whisper: '+ model_size +'\n')
ft.write('- Idioma: '+ language +'\n')
ft.write('- Numero hablantes: '+ str(num_speakers) +'\n')

for (i, segment) in enumerate(segments_group):
  if i == 0 or segments_group[i - 1]["speaker"] != segment["speaker"]:
    #f.write("\n" + segment["speaker"] + ' ClipTime: ' + str(time(segment["start"])))
    ft.write("\n" + segment["speaker"] + '    Time: ' + str(time(segment["start"])) + '\n')
  ft.write(segment["text"][1:] + ' ')

ft.close()

In [ ]:
#!zip -r outs.zip texts clips
#!zip -r out_texts.zip texts
files.download(direction_text+"translation.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# HTML Generator


In [ ]:
ft = open(direction_text+"translation.html", "w", encoding='utf-8')
ft.write('<!DOCTYPE html><html lang="es"><head> <meta charset="UTF-8"> <meta name="viewport" content="width=device-width, initial-scale=1.0"><title>Document</title></head><body>')
ft.write('<h3>Atencion: Documento transcrito por secciones e IA, Revisar</h3>\n')
ft.write('<h3>Datos:</h3>\n')
ft.write('<ul>\n')
ft.write(f'<li>Version Transcriptior: v{version_tscript}</li>\n')
ft.write('<li>Nombre archivo original: '+ nombre_archivo +'</li>\n')
ft.write('<li>Duracion de la grabacion: '+ str(time(duration_total)) +'</li>\n')
ft.write('<li>Duracion maxima de cada clip: '+ str(time(clips_time_min * 60)) +'</li>\n')
ft.write('<li>Tamaño del modelo de Whisper: '+ model_size +'</li>\n')
ft.write('<li>Idioma: '+ language +'</li>\n')
ft.write('<li>Numero hablantes: '+ str(num_speakers) +'</li>\n')
ft.write('</ul><p>')

for (i, segment) in enumerate(segments_group):
  if i == 0 or segments_group[i - 1]["speaker"] != segment["speaker"]:
    ft.write("\n</p><p>" + segment["speaker"] + '    Time: ' + str(time(segment["start"])) + '</p>\n<p>')
  ft.write('<span style="color: orange;">[SP.{}={}-{}]</span> '.format(segment["speaker"][-1], str(time(segment["start"])), str(time(segment["end"])))+segment["text"][1:] + ' ')
ft.write('</p></body></html>')

ft.close()

In [ ]:

files.download(direction_text+"translation.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Iniciar retranscripcion partida atomica

In [ ]:
print(duration)

583.5946666666666


Fin de retranscripcion partida atomica

In [ ]:

#files.download(direction_text+"translation.html")